In [14]:
import os 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

In [15]:
## preprocess --> blood-gas, blood-gas-arterial, labs, urine-output, vitals, vasopressors
folder = '../sepsis3_queries/data'
sepsis3 = pd.read_csv(f'{folder}/sepsis-df-3.csv')
blood_gas = pd.read_csv(f'{folder}/blood-gas-df-3.csv')
blood_gas_arterial = pd.read_csv(f'{folder}/blood-gas-arterial-df-3.csv')
labs = pd.read_csv(f'{folder}/labs-df-3.csv')
urine_output = pd.read_csv(f'{folder}/urine-output-df-3.csv')
vitals = pd.read_csv(f'{folder}/vitals-df-3.csv')
vasopressors = pd.read_csv(f'{folder}/vasopressors-ts-df-3.csv')

In [16]:
blood_gas_arterial.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'specimen',
       'specimen_pred', 'specimen_prob', 'so2', 'spo2', 'po2', 'pco2',
       'fio2_chartevents', 'fio2', 'aado2', 'aado2_calc', 'pao2fio2', 'ph',
       'baseexcess', 'bicarbonate', 'totalco2', 'hematocrit', 'hemoglobin',
       'carboxyhemoglobin', 'methemoglobin', 'chloride', 'calcium',
       'temperature', 'potassium', 'sodium', 'lactate', 'glucose', 'intubated',
       'tidalvolume', 'ventilationrate', 'ventilator', 'peep', 'o2flow',
       'requiredo2'],
      dtype='object')

In [17]:
blood_gas.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'specimen', 'aado2',
       'baseexcess', 'bicarbonate', 'totalco2', 'carboxyhemoglobin',
       'chloride', 'calcium', 'glucose', 'hematocrit', 'hemoglobin',
       'intubated', 'lactate', 'methemoglobin', 'o2flow', 'fio2', 'so2',
       'pco2', 'peep', 'ph', 'po2', 'potassium', 'requiredo2', 'sodium',
       'temperature', 'tidalvolume', 'ventilationrate', 'ventilator'],
      dtype='object')

In [18]:
bg = blood_gas.set_index('icustay_id')
bga = blood_gas_arterial.set_index('icustay_id')
c1 = pd.concat([bg, bga])

c1[c1.index == 200075][['charttime','po2']]

,charttime,po2
icustay_id,,
200075,2159-09-23 03:18:00,NaN
200075,2159-09-23 07:55:00,114.0
200075,2159-09-23 08:27:00,NaN
200075,2159-09-23 14:37:00,NaN
200075,2159-09-23 17:30:00,NaN
200075,2159-09-23 07:55:00,114.0


In [19]:
c1 = c1.drop_duplicates(subset=['subject_id','charttime'], keep="first").drop(['subject_id', 'hadm_id'], axis=1)


In [20]:
c1[c1.index == 200075][['charttime','po2']]

,charttime,po2
icustay_id,,
200075,2159-09-23 03:18:00,NaN
200075,2159-09-23 07:55:00,114.0
200075,2159-09-23 08:27:00,NaN
200075,2159-09-23 14:37:00,NaN
200075,2159-09-23 17:30:00,NaN


In [21]:
vt = vitals.set_index('icustay_id').drop(['subject_id', 'hadm_id'], axis=1)
lb = labs.set_index('icustay_id').drop(['subject_id', 'hadm_id'], axis=1)
c2 = pd.concat([vt, lb])
c2 = c2.sort_values(by=['icustay_id', 'charttime'])
c2[c2.index == 200075]
#[['charttime','po2']]

,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,aniongap,...,hemoglobin,lactate,platelet,potassium,ptt,inr,pt,sodium,bun,wbc
icustay_id,,,,,,,,,,,,,,,,,,,,,
200075,2159-09-23 01:00:00,NaN,NaN,NaN,NaN,NaN,36.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:34:00,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:34:00,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:35:00,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:35:00,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200075,2159-09-24 00:00:00,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-24 00:00:00,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-24 00:00:00,NaN,127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
vasopressors = vasopressors.rename(columns={'starttime':'charttime'}).drop(['intime', 'endtime'], axis=1)
vs = vasopressors.set_index('icustay_id')
#set_index('icustay_id')
uo = urine_output.set_index('icustay_id').drop(['subject_id', 'hadm_id'], axis=1)
c3 = pd.concat([vs, uo], axis=0)
c3 = c3.sort_values(by=['icustay_id', 'charttime'])
c3[c3.index == 200075]

,charttime,dobutamine_dose,dopamine_dose,vasopressin_dose,phenylephrine_dose,epinephrine_dose,norepinephrine_dose,urineoutput
icustay_id,,,,,,,,
200075,2159-09-23 01:40:00,NaN,NaN,NaN,NaN,NaN,0.339943,NaN
200075,2159-09-23 01:58:00,NaN,NaN,NaN,NaN,NaN,NaN,450.0
200075,2159-09-23 02:25:00,NaN,NaN,NaN,NaN,NaN,0.183005,NaN
200075,2159-09-23 02:46:00,NaN,NaN,NaN,NaN,NaN,0.669275,NaN
200075,2159-09-23 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,60.0
200075,2159-09-23 03:50:00,NaN,NaN,NaN,NaN,NaN,0.801598,NaN
200075,2159-09-23 04:59:00,NaN,NaN,NaN,NaN,NaN,0.181161,NaN
200075,2159-09-23 05:12:00,NaN,NaN,NaN,NaN,NaN,0.798591,NaN
200075,2159-09-23 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,60.0


In [24]:
time_series =  pd.concat([c1, c2, c3])
time_series = time_series.sort_values(by=['icustay_id', 'charttime'])
time_series[time_series.index == 200075]

,charttime,specimen,aado2,baseexcess,bicarbonate,totalco2,carboxyhemoglobin,chloride,calcium,glucose,...,pt,bun,wbc,dobutamine_dose,dopamine_dose,vasopressin_dose,phenylephrine_dose,epinephrine_dose,norepinephrine_dose,urineoutput
icustay_id,,,,,,,,,,,,,,,,,,,,,
200075,2159-09-23 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-23 01:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200075,2159-09-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200075,2159-09-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
time_series.columns

Index(['charttime', 'specimen', 'aado2', 'baseexcess', 'bicarbonate',
       'totalco2', 'carboxyhemoglobin', 'chloride', 'calcium', 'glucose',
       'hematocrit', 'hemoglobin', 'intubated', 'lactate', 'methemoglobin',
       'o2flow', 'fio2', 'so2', 'pco2', 'peep', 'ph', 'po2', 'potassium',
       'requiredo2', 'sodium', 'temperature', 'tidalvolume', 'ventilationrate',
       'ventilator', 'specimen_pred', 'specimen_prob', 'spo2',
       'fio2_chartevents', 'aado2_calc', 'pao2fio2', 'heartrate', 'sysbp',
       'diasbp', 'meanbp', 'resprate', 'tempc', 'aniongap', 'albumin', 'bands',
       'bilirubin', 'creatinine', 'platelet', 'ptt', 'inr', 'pt', 'bun', 'wbc',
       'dobutamine_dose', 'dopamine_dose', 'vasopressin_dose',
       'phenylephrine_dose', 'epinephrine_dose', 'norepinephrine_dose',
       'urineoutput'],
      dtype='object')

In [25]:
time_series.reset_index(inplace=True)
folder = 'data_stage_1'
time_series.to_csv(os.path.join(folder, 'time_series.csv'),sep=',',index=False)